In [14]:
!conda install -c roebel mido

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - mido

Current channels:

  - https://conda.anaconda.org/roebel/win-64
  - https://conda.anaconda.org/roebel/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [6]:
!python -V

Python 3.9.16


In [12]:
!pip install mido

     -------------------------------------- 51.1/51.1 kB 869.2 kB/s eta 0:00:00


#

In [213]:
import mido
import json
import copy

Перевод MIDI в JSON.
# длительности на данный момент постоянные

In [214]:
def mid_to_json(mid):
    temp=[]
    new_data={'res':{}}
    new_data.get('res')['notes']=[]
    for i in mid.tracks[0]:
        if i.type=='note_on':
            new_data.get('res')['notes'].append({'on':i.note})
        elif i.type=='sysex':
            new_data.get('res')['notes'].append({'sysex':i.time}) 
        elif i.type=='note_off':
            new_data.get('res')['notes'].append({'off':i.note})
    return(new_data)

Входной JSON: подразумевается, что помимо res мгут подаваться какие-либо ещё данные
# возможно settings, где будет определяться тональность пользователем

In [240]:
json_string = """
{
    "res": {
        "notes": [
            36,
            36,
            38,
            38,
            39,
            39,
            41,
            39,
            39,
            38,
            38,
            36,
            36,
            34,
            36,
            36
        ],
        "chords": [
            [36,40,43,47],
            [38,42,45,49]
        ]
    }
}
"""
 
data = json.loads(json_string)

In [241]:
data.get('res').get('notes')

[36, 36, 38, 38, 39, 39, 41, 39, 39, 38, 38, 36, 36, 34, 36, 36]

Для того чтобы ноты не отжимались мгновенно, нужно задать time (в случае нажимания и отпускания нескольких клавиш одновременно, мы применяли системное сообщение с параметром time между циклами из нажатия и отпускания клавиш)

In [242]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

track.append(mido.Message('program_change', program=12, time=0))
j=-1
for i in data.get('res').get('notes'):
    track.append(mido.Message('note_on', note=i))
    track.append(mido.Message('note_off', note=i, time=240))

mid.save('new_song.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 36},
   {'off': 36},
   {'on': 36},
   {'off': 36},
   {'on': 38},
   {'off': 38},
   {'on': 38},
   {'off': 38},
   {'on': 39},
   {'off': 39},
   {'on': 39},
   {'off': 39},
   {'on': 41},
   {'off': 41},
   {'on': 39},
   {'off': 39},
   {'on': 39},
   {'off': 39},
   {'on': 38},
   {'off': 38},
   {'on': 38},
   {'off': 38},
   {'on': 36},
   {'off': 36},
   {'on': 36},
   {'off': 36},
   {'on': 34},
   {'off': 34},
   {'on': 36},
   {'off': 36},
   {'on': 36},
   {'off': 36}]}}

эмпирически time - длительность, velocity - громкость

Наиболее популярные аккорды: мажорное/минорное трезвучия, мажорный/минорный септаккорды. Для определения мажорного/минорного необходимо от тоники взять ноту на 4/3 полутона выше и на 3/4 полутона выше найденной ноты. Для септаккордов берётся 4-ая нота на 4/3 выше предыдущей.

# описан только случай когда тоника является самой низкой нотой (т.е. C-E-G и G-C-E оба являются До-мажором, но в списке указан только первый вариант)

In [243]:
major=[]
minor=[]
major7=[]
minor7=[]
for i in range(0,127-4-3-4):
    major.append([i, i+4, i+4+3])
    minor.append([i, i+3, i+3+4])
    major7.append([i, i+4, i+4+3, i+4+3+4])
    minor7.append([i, i+3, i+3+4, i+3+4+3])

In [244]:
major[0]

[0, 4, 7]

Полученные на вход ноты преобразуются в аккорды

In [248]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

for i in data.get('res').get('notes'):
    for j in major7[i]:
        track.append(mido.Message('note_on', note=j))
    track.append(mido.Message('sysex', time=250))
    for j in major7[i]:
        track.append(mido.Message('note_off', note=j))

mid.save('new_song_chords.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 47},
   {'sysex': 250},
   {'off': 36},
   {'off': 40},
   {'off': 43},
   {'off': 47},
   {'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 47},
   {'sysex': 250},
   {'off': 36},
   {'off': 40},
   {'off': 43},
   {'off': 47},
   {'on': 38},
   {'on': 42},
   {'on': 45},
   {'on': 49},
   {'sysex': 250},
   {'off': 38},
   {'off': 42},
   {'off': 45},
   {'off': 49},
   {'on': 38},
   {'on': 42},
   {'on': 45},
   {'on': 49},
   {'sysex': 250},
   {'off': 38},
   {'off': 42},
   {'off': 45},
   {'off': 49},
   {'on': 39},
   {'on': 43},
   {'on': 46},
   {'on': 50},
   {'sysex': 250},
   {'off': 39},
   {'off': 43},
   {'off': 46},
   {'off': 50},
   {'on': 39},
   {'on': 43},
   {'on': 46},
   {'on': 50},
   {'sysex': 250},
   {'off': 39},
   {'off': 43},
   {'off': 46},
   {'off': 50},
   {'on': 41},
   {'on': 45},
   {'on': 48},
   {'on': 52},
   {'sysex': 250},
   {'off': 41},
   {'off': 45},
   {'off': 48},

Преобразование входных аккордов в MIDI и обратно

In [223]:
data.get('res').get('chords')

[[36, 40, 43, 47], [38, 42, 45, 49]]

In [249]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

for i in data.get('res').get('chords'):
    for j in i:
        track.append(mido.Message('note_on', note=j))
    track.append(mido.Message('sysex', time=250))
    for j in i:
        track.append(mido.Message('note_off', note=j))

mid.save('USER_song_chords.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 47},
   {'sysex': 250},
   {'off': 36},
   {'off': 40},
   {'off': 43},
   {'off': 47},
   {'on': 38},
   {'on': 42},
   {'on': 45},
   {'on': 49},
   {'sysex': 250},
   {'off': 38},
   {'off': 42},
   {'off': 45},
   {'off': 49}]}}

In [225]:
#new_data=copy.deepcopy(data)
#
#new_data.get('res')['new_chords']=[major[i] for i in data.get('res').get('notes')]
#json.dumps(new_data)

'{"res": {"notes": [36, 36, 38, 38, 39, 39, 41, 39, 39, 38, 38, 36, 36, 34, 36, 36], "chords": [[36, 40, 43, 47], [38, 42, 45, 49]], "new_chords": [[36, 40, 43], [36, 40, 43], [38, 42, 45], [38, 42, 45], [39, 43, 46], [39, 43, 46], [41, 45, 48], [39, 43, 46], [39, 43, 46], [38, 42, 45], [38, 42, 45], [36, 40, 43], [36, 40, 43], [34, 38, 41], [36, 40, 43], [36, 40, 43]]}}'

I — IV — V — V.
I — I — IV — V.
I — IV — I — V.
I — IV — V — IV.

.
.
.

Блюзовая гармония

I — I — I — I
IV — IV — I — I
V — IV — I — I

C-C-C-C-F-F-C-C-G-F-C-C

i i i i i+5 i+5 i i i+7 i+5 i i

In [226]:
chord3_p=[0,5,7,7,0,0,5,7,0,5,0,7,0,5,7,5]
blues_p=[0,0,0,0,5,5,0,0,7,5,0,0]

Построение нотной прогрессии от одной ноты

In [227]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

for i in [76]:
    for j in chord3_p:
        track.append(mido.Message('note_on', note=i+j))
        track.append(mido.Message('note_off', note=i+j, time=240))

mid.save('new_song_progression.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 76},
   {'off': 76},
   {'on': 81},
   {'off': 81},
   {'on': 83},
   {'off': 83},
   {'on': 83},
   {'off': 83},
   {'on': 76},
   {'off': 76},
   {'on': 76},
   {'off': 76},
   {'on': 81},
   {'off': 81},
   {'on': 83},
   {'off': 83},
   {'on': 76},
   {'off': 76},
   {'on': 81},
   {'off': 81},
   {'on': 76},
   {'off': 76},
   {'on': 83},
   {'off': 83},
   {'on': 76},
   {'off': 76},
   {'on': 81},
   {'off': 81},
   {'on': 83},
   {'off': 83},
   {'on': 81},
   {'off': 81}]}}

Применение квинтового круга для определения наиболее созвучных аккордов

In [228]:
circle_of_fifths=[0]
for i in range(0, 11):
    circle_of_fifths.append((circle_of_fifths[-1]+7)%12)
print(circle_of_fifths)

[0, 7, 2, 9, 4, 11, 6, 1, 8, 3, 10, 5]


In [229]:
for i in range(0,len(circle_of_fifths)):
    print(circle_of_fifths[i-1],circle_of_fifths[i],circle_of_fifths[(i+1)%len(circle_of_fifths)])

5 0 7
0 7 2
7 2 9
2 9 4
9 4 11
4 11 6
11 6 1
6 1 8
1 8 3
8 3 10
3 10 5
10 5 0


In [230]:
print(major[circle_of_fifths[i-1]],major[circle_of_fifths[i]],major[circle_of_fifths[(i+1)%len(circle_of_fifths)]])
print(minor[circle_of_fifths[i-1-9]],minor[circle_of_fifths[i-9]],minor[circle_of_fifths[i+1-9]])

[10, 14, 17] [5, 9, 12] [0, 4, 7]
[7, 10, 14] [2, 5, 9] [9, 12, 16]


Перебор близких аккордов по одной ноте (тональности)

In [231]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

for i in [60]:
    for j in [major[circle_of_fifths[i%12-1]],major[circle_of_fifths[i%12]],major[circle_of_fifths[(i%12+1)%len(circle_of_fifths)]],
              minor[circle_of_fifths[i%12+1-9]],minor[circle_of_fifths[i%12-9]],minor[circle_of_fifths[i%12-1-9]]]:
        for k in j:
            track.append(mido.Message('note_on', note=i+k))
        track.append(mido.Message('sysex', time=500))
        for k in j:    
            track.append(mido.Message('note_off', note=i+k))

mid.save('new_circle_progression.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 65},
   {'on': 69},
   {'on': 72},
   {'sysex': 500},
   {'off': 65},
   {'off': 69},
   {'off': 72},
   {'on': 60},
   {'on': 64},
   {'on': 67},
   {'sysex': 500},
   {'off': 60},
   {'off': 64},
   {'off': 67},
   {'on': 67},
   {'on': 71},
   {'on': 74},
   {'sysex': 500},
   {'off': 67},
   {'off': 71},
   {'off': 74},
   {'on': 64},
   {'on': 67},
   {'on': 71},
   {'sysex': 500},
   {'off': 64},
   {'off': 67},
   {'off': 71},
   {'on': 69},
   {'on': 72},
   {'on': 76},
   {'sysex': 500},
   {'off': 69},
   {'off': 72},
   {'off': 76},
   {'on': 62},
   {'on': 65},
   {'on': 69},
   {'sysex': 500},
   {'off': 62},
   {'off': 65},
   {'off': 69}]}}

# тональность определяется первой нотой в данных пользователя, а количество нот мелодии до смены аккорда задано числом

In [232]:
#TODO
def get_melody_notes_per_chord():
    return 4

In [233]:
def get_key(notes):
    return notes[0]

In [251]:
json_string = """
{
    "res": {
        "notes": [
            60,
            60,
            67,
            67,
            69,
            69,
            67,
            65,
            65,
            64,
            64,
            62,
            62,
            60
        ]
    }
}
"""
 
data = json.loads(json_string)

Построение аккордовой прогрессии по мелодии (аккордом берётся мажорный аккорд каждой 4-ой ноты мелодии)

In [255]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

notes = data.get('res').get('notes')
notes_cycle = get_melody_notes_per_chord()
notes_cycle_len = len(notes)//notes_cycle+1 #4
for i in range(0,notes_cycle_len):
    start=i*notes_cycle
    end=(i+1)*notes_cycle
    if end > len(notes):
        end=len(notes)
    cycle_notes=notes[start:end]
    melody_key = get_key(cycle_notes)
    for k in major[melody_key]:
        track.append(mido.Message('note_on', note=k-24))
    for j in cycle_notes:
        track.append(mido.Message('note_on', note=j))
        track.append(mido.Message('sysex', time=500))
        track.append(mido.Message('note_off', note=j))
    for k in major[melody_key]:
        track.append(mido.Message('note_off', note=k-24))

mid.save('new_accompany_melody_progression.mid')
mid_to_json(mid)

{'res': {'notes': [{'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 60},
   {'sysex': 500},
   {'off': 60},
   {'on': 60},
   {'sysex': 500},
   {'off': 60},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'off': 36},
   {'off': 40},
   {'off': 43},
   {'on': 45},
   {'on': 49},
   {'on': 52},
   {'on': 69},
   {'sysex': 500},
   {'off': 69},
   {'on': 69},
   {'sysex': 500},
   {'off': 69},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'on': 65},
   {'sysex': 500},
   {'off': 65},
   {'off': 45},
   {'off': 49},
   {'off': 52},
   {'on': 41},
   {'on': 45},
   {'on': 48},
   {'on': 65},
   {'sysex': 500},
   {'off': 65},
   {'on': 64},
   {'sysex': 500},
   {'off': 64},
   {'on': 64},
   {'sysex': 500},
   {'off': 64},
   {'on': 62},
   {'sysex': 500},
   {'off': 62},
   {'off': 41},
   {'off': 45},
   {'off': 48},
   {'on': 38},
   {'on': 42},
   {'on': 45},
   {'on': 62},
   {'sysex': 500},
   {'off': 62},
   {'on': 

Построение аккордовой прогрессии по мелодии (аккордом берётся рандомный созвучный аккорд тональности мелодии)

In [257]:
import random

mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)

notes = data.get('res').get('notes')
notes_cycle = get_melody_notes_per_chord()
notes_cycle_len = len(notes)//notes_cycle+1 #4
melody_key = get_key(notes)
for i in range(0,notes_cycle_len):
    start=i*notes_cycle
    end=(i+1)*notes_cycle
    if end > len(notes):
        end=len(notes)
    cycle_notes=notes[start:end]
    random_chord = random.choice([major[circle_of_fifths[melody_key%12]],
              major[circle_of_fifths[melody_key%12-1]],
              major[circle_of_fifths[(melody_key%12+1)%len(circle_of_fifths)]],
              minor[circle_of_fifths[melody_key%12+1-9]],
              minor[circle_of_fifths[melody_key%12-9]],
              minor[circle_of_fifths[melody_key%12-1-9]]])
    print(random_chord)
    for k in random_chord:
        track.append(mido.Message('note_on', note=melody_key+k-24))
    for j in cycle_notes:
        track.append(mido.Message('note_on', note=j))
        track.append(mido.Message('sysex', time=500))
        track.append(mido.Message('note_off', note=j))
    for k in random_chord:
        track.append(mido.Message('note_off', note=melody_key+k-24))

mid.save('new_accompany_melody_circle_progression.mid')

[9, 12, 16]
[0, 4, 7]
[9, 12, 16]
[0, 4, 7]


In [258]:
mid_to_json(mid)

{'res': {'notes': [{'on': 45},
   {'on': 48},
   {'on': 52},
   {'on': 60},
   {'sysex': 500},
   {'off': 60},
   {'on': 60},
   {'sysex': 500},
   {'off': 60},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'off': 45},
   {'off': 48},
   {'off': 52},
   {'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 69},
   {'sysex': 500},
   {'off': 69},
   {'on': 69},
   {'sysex': 500},
   {'off': 69},
   {'on': 67},
   {'sysex': 500},
   {'off': 67},
   {'on': 65},
   {'sysex': 500},
   {'off': 65},
   {'off': 36},
   {'off': 40},
   {'off': 43},
   {'on': 45},
   {'on': 48},
   {'on': 52},
   {'on': 65},
   {'sysex': 500},
   {'off': 65},
   {'on': 64},
   {'sysex': 500},
   {'off': 64},
   {'on': 64},
   {'sysex': 500},
   {'off': 64},
   {'on': 62},
   {'sysex': 500},
   {'off': 62},
   {'off': 45},
   {'off': 48},
   {'off': 52},
   {'on': 36},
   {'on': 40},
   {'on': 43},
   {'on': 62},
   {'sysex': 500},
   {'off': 62},
   {'on': 